In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import *
import numpy as np
from keras.utils import to_categorical

Using TensorFlow backend.
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
(train_data,test_data) = mnist.load_data()

In [4]:
x_train = train_data[0]
x_train = np.asarray([np.reshape(data,(28,28,1)) for data in x_train])
y_train = to_categorical(train_data[1])

In [6]:
x_test = test_data[0]
x_test = np.asarray([np.reshape(data,(28,28,1)) for data in x_test])
y_test_pure = test_data[1]
y_test = to_categorical(test_data[1])

In [7]:
def make_conv_model_list():
    model = Sequential()
    model.add(Conv2D(32,(3,3),use_bias=True,activation='relu',padding='same',input_shape=(28,28,1)))
    model.add(MaxPool2D(2,2))
    model.add(ZeroPadding2D(padding=(1, 1), data_format=None))
    model.add(Conv2D(64,(3,3),use_bias=True,padding='same',activation='relu'))
    model.add(MaxPool2D(2,2))
    model.add(Flatten())
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['acc'])
    model.summary()
    return model

In [9]:
#make  ensemble list 5

ensemble_model_list = []
ensemble_size = 5

In [10]:
for i in range(ensemble_size):
    this_model = make_conv_model_list()
    print('i_th model training')
    this_model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))
    
    ensemble_model_list.append(this_model)

W0314 19:02:21.358536  1200 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0314 19:02:21.375535  1200 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0314 19:02:21.379586  1200 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0314 19:02:21.401076  1200 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0314 19:02:21.443095  1200 deprecation_wrapper.py:119] From C:\Users\Admin\Ana

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 45s 742us/step - loss: 0.9290 - acc: 0.8619 - val_loss: 0.0679 - val_acc: 0.9800
Epoch 2/5
60000/60000 [==============================] - 44s 735us/step - loss: 0.1198 - acc: 0.9655 - val_loss: 0.0598 - val_acc: 0.9806
Epoch 3/5
60000/60000 [==============================] - 44s 735us/step - loss: 0.0797 - acc: 0.9768 - val_loss: 0.0416 - val_acc: 0.9872
Epoch 4/5
60000/60000 [==============================] - 44s 737us/step - loss: 0.0637 - acc: 0.9811 - val_loss: 0.0416 - val_acc: 0.9860
Epoch 5/5
60000/60000 [==============================] - 44s 725us/step - loss: 0.0506 - acc: 0.9846 - val_loss: 0.0422 - val_acc: 0.9872
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 28, 28, 32)        320       
________________________________________

In [11]:
#results of all models
    
all_predicted = np.zeros(y_test.shape)

In [12]:
for one in ensemble_model_list:
    all_predicted =all_predicted+one.predict(x_test)

In [13]:
#get final res

final_res = []
for one in all_predicted:
    final_res.append(np.argmax(one))

In [14]:
final_res = np.asarray(final_res)

In [15]:
entire = y_test_pure.shape[0]

In [16]:
ele = 0
for one in range(entire):
    if(y_test_pure[one]==final_res[one]):
        ele=ele+1
    
res_acc = ele/entire

In [17]:
print('fianl result test accuracy:',res_acc)

fianl result test accuracy: 0.9918
